### TD n°1 Deep Learning - Exo 1 FizzBuzz

In [1]:
import numpy as np
import torch

On modélise le pb commme un pb de classification multi-classe.

Les sorties possibles 0 / 1 / 2 / 3 correspondent respectivement 
* au nombre lui-même, 
* à fizz pour les multiples de 3, 
* buzz pour les multiples de 5,
* fizzbuzz pour les multiples de 15.

###  Modélisation sortie de la vérité terrain

In [10]:
def fizz_buzz (i, prediction) :
    return [str(i),"fizz","buzz","fizzbuzz"][prediction]

Création de la vérité terrain : [number, fizz, buzz, fizzbuzz]


In [2]:
def fizz_buzz_encode(i):
    if i % 15 == 0:
        return 3
    elif i % 5 == 0:
        return 2
    elif i % 3 == 0:
        return 1
    else:
        return 0
    raise NotImplementedError

Testez vos fonctions en vérifiant que vous renvoyez la bonne sortie pour $i=1,\dots,19$

In [ ]:
for i in range(1, 20): 
    print(
        i, 
        fizz_buzz_encode(i),# classe  
        fizz_buzz(i , fizz_buzz_encode(i)) # classe -> résultat i ? fizz ? buzz ? fizzbuzz ?
        )

### Modélisation des entrées et des données

Encodage binaire 


In [3]:
NUM_DIGITS = 10
def binary_encode (i , num_digits=NUM_DIGITS ) :
    return [i >> d & 1 for d in range(num_digits)] #TO DO. Prend un nombre i et le convertit en binaire

In [4]:
start_train, end_train = 101,2**NUM_DIGITS # 101, 1024
start_test, end_test = 1, 101

# train
X_train = torch.FloatTensor ([ binary_encode (i , NUM_DIGITS ) for i in range (start_train, end_train)])
Y_train = torch.LongTensor ([fizz_buzz_encode(i) for i in range(start_train, end_train)]).squeeze()

# test
X_test = torch.FloatTensor ([ binary_encode (i , NUM_DIGITS) for i in range (start_test, end_test) ])
print(X_test[:5])

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])


### Création du modèle, de la loss et de l'optimiseur

In [6]:
# nombre de neurones dans la couche cachée
NUM_HIDDEN = 100

# définition du MLP à 1 couche cachée (non linearite ReLU)
model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS, NUM_HIDDEN), # Entrée -> 10 digits
    torch.nn.ReLU(), #Non linéarité ReLU
    torch.nn.Linear(NUM_HIDDEN, 4) # 4 classes -> sortie
    )

__Fonction de coût__ 

CrossEntropyLoss vs NLLLoss ? (negative log likelihood)

CE loss : The input is expected to contain the unnormalized logits for each class, *which do not need to be positive or sum to 1, in general*. 

NLLLoss : il faut avoir calculé un softmax

https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss 

In [ ]:
# Cross-entropy formule générique que l'on utilise tout le temps. Elle dépend de la forme exacte de la sortie du réseau de neurones 
# Sigmoid pour problèmes binaires
# Softmax pour problèmes multi-classes
# On pourrait en soi avoir une sigmoid par classe, mais on préfère softmax comme on est exclusif
# NLLL : Negative Log Likelihood Loss

In [7]:
# fonction de coût
loss_fn = torch.nn.CrossEntropyLoss()   

In [8]:
# optimiseur --> choix du learning rate
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05) # Mettre à jour tous les paramètres du modèle
# Hyperparamètre important à cette étape: le learning rate

### Apprentissage

In [ ]:
# On doit toujours avoir un ensemble de train (paramètres), un ensemble de val (différents hyperparamètres - learning rate, nombre de couches, etc) 
# et un ensemble de test (pour la généralisation), on monitore si ça se passe bien sur val
# on test une fois sur un ensemble de test (pour la généralisation)

In [11]:
BATCH_SIZE = 128

raw_data_test = np.arange(1, 101) # valeurs de test

for epoch in range(10000):
    for start in range(0, len(X_train), BATCH_SIZE):
        end = start + BATCH_SIZE
        batchX = X_train[start:end] # Pas de problème au bord de la liste, on peut dépasser la taille de la liste en slicing
        batchY = Y_train[start:end]

        # prediction et calcul de la loss
        y_pred = model(batchX)
        loss = loss_fn(y_pred, batchY)
    
        # mettre les gradients à 0 avant la passe retour (backward)
        optimizer.zero_grad()
    
        # rétro-propagation
        loss.backward()
        optimizer.step()

    # calcul coût  (et affichage)
    loss = loss_fn( model(X_train), Y_train)
    if epoch%100 == 0:
        print('epoch {} training loss {}'.format(epoch, round(loss.item(), 3)))

    # visualisation des résultats en cours d'apprentissage
    if(epoch%1000==0):
        Y_test_pred = model(X_test)
        val, idx = torch.max(Y_test_pred,1)
        ii=idx.data.numpy()
        # numbers = np.arange(1, 101)
        output = np.vectorize(fizz_buzz)(raw_data_test, ii)
        print(output)

epoch 0 training loss 1.157
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100']
epoch 100 training loss 1.132
epoch 200 training loss 1.119
epoch 300 training loss 1.097
epoch 400 training loss 1.062
epoch 500 training loss 1.006
epoch 600 training loss 0.932
epoch 700 training loss 0.856
epoch 800 training loss 0.779
epoch 900 training loss 0.709
epoch 1000 training loss 0.63
['1' 'fizz' 'fizz' '4' '5' 'fizz' '7' '8' 'fizz' '10' '11' 'fizz' '13'
 '14' '15' '16' '17' 'fizz' '19' '20' '21' '22' '23' 'fizz' '25' '26'
 'fizz' '28' '29' '30' '31' '32

__Affichage des résultats__

In [12]:
# Sortie finale (calcul lisible)
Y_test_pred = model(X_test)
val, idx = torch.max(Y_test_pred,1)
ii=idx.data.numpy()
output = np.vectorize(fizz_buzz)(raw_data_test, ii)
print("============== Final result ============")
print(output)

# Sortie finale (calcul plus compact des predictions)
Y_test_pred = model(X_test)
predictions = zip(range(1, 101), list(Y_test_pred.max(1)[1].data.tolist()))
print("============== Final result ============")
print ([fizz_buzz(i, x) for (i, x) in predictions])

============== Final result ============
['1' 'fizz' 'fizz' '4' '5' 'fizz' '7' '8' 'fizz' 'buzz' '11' 'fizz' '13'
 '14' 'fizzbuzz' '16' '17' 'fizz' '19' 'buzz' '21' '22' '23' 'fizz' '25'
 '26' 'fizz' '28' '29' 'fizzbuzz' '31' '32' 'fizz' 'buzz' 'buzz' 'fizz'
 '37' '38' 'fizz' 'buzz' '41' 'fizz' '43' '44' 'fizzbuzz' '46' '47' 'fizz'
 '49' 'buzz' 'fizz' '52' '53' 'fizz' 'buzz' '56' 'fizz' '58' '59'
 'fizzbuzz' 'fizz' '62' 'fizz' '64' '65' 'fizz' '67' '68' '69' '70' '71'
 'fizz' '73' '74' 'fizzbuzz' '76' '77' 'fizz' '79' 'buzz' '81' '82' '83'
 'fizz' 'buzz' '86' '87' '88' '89' 'fizzbuzz' '91' 'buzz' 'fizz' '94'
 'buzz' 'fizz' '97' '98' 'fizz' 'buzz']
============== Final result ============
['1', 'fizz', 'fizz', '4', '5', 'fizz', '7', '8', 'fizz', 'buzz', '11', 'fizz', '13', '14', 'fizzbuzz', '16', '17', 'fizz', '19', 'buzz', '21', '22', '23', 'fizz', '25', '26', 'fizz', '28', '29', 'fizzbuzz', '31', '32', 'fizz', 'buzz', 'buzz', 'fizz', '37', '38', 'fizz', 'buzz', '41', 'fizz', '43', '44

__Calcul des performances (classification accuracy)__

In [23]:
gtY = np.array([fizz_buzz_encode(i) for i in raw_data_test])  # ground truth
print("test perf: ", np.mean(gtY == np.array(Y_test_pred)))

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

__Avec un validation set !__

In [ ]:
NUM_VAL = 100
p = np.random.permutation(range(len(X_train)))
# permettre de mélanger les nombres
X_train, Y_train = X_train[..., :], Y_train[...]
X_val, Y_val = X_train[..., :], Y_train[...]
X_train, Y_train = X_train[...:, :], Y_train[...]

In [ ]:
BATCH_SIZE = 128
NUM_HIDDEN = 100

model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS, NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN, 4)
    )

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05)

for epoch in range(10000):
    for start in range(0, len(X_train), BATCH_SIZE):
        end = start + BATCH_SIZE
        batchX = X_train[start:end]
        batchY = Y_train[start:end]

        # prediction et calcul de la loss
        y_pred = model(batchX)
        loss = loss_fn(y_pred, batchY)
    
        # mettre les gradients à 0 avant la passe retour (backward)
        optimizer.zero_grad()
    
        # rétro-propagation
        loss.backward()
        optimizer.step()

    # calcul coût  (et affichage)
    loss = loss_fn( model(X_train), Y_train)
    if epoch%100 == 0:
        print('epoch {} training loss {}'.format(epoch, round(loss.item(), 3)))

    # visualisation des résultats en cours d'apprentissage
    # cette fois-ci sur l'ensemble de validation
    if(epoch%1000==0):
        # train acc
        Y_train_pred = model(X_train)
        print("train perf: ", np.mean(Y_train.data.numpy() == Y_train_pred.max(1)[1].data.numpy() ) )
        Y_val_pred = model(X_val)
        print("val perf: ", np.mean(Y_val.data.numpy() == Y_val_pred.max(1)[1].data.numpy() ) )

In [ ]:
# Sortie finale (calcul lisible)
Y_test_pred = model(X_test)
val, idx = torch.max(Y_test_pred,1)
ii=idx.data.numpy()
output = np.vectorize(fizz_buzz)(raw_data_test, ii)
print("============== Final result ============")
print(output)

# Sortie finale (calcul plus compact des predictions)
Y_test_pred = model(X_test)
predictions = zip(range(1, 101), list(Y_test_pred.max(1)[1].data.tolist()))
print("============== Final result ============")
print ([fizz_buzz(i, x) for (i, x) in predictions])

__Experiment with different settings__

* learning rate
* optimizer
* scheduler
* number of training samples
* architecture of the MLP (number of hidden units)
* number of epochs
* ...